In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout,Activation,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import SGD
from sklearn.svm import *
from sklearn.linear_model import LogisticRegression


# #Notif sound
# from google.colab import output
# def notif():
#   output.eval_js('new Audio("https://notificationsounds.com/message-tones/appointed-529/download/mp3").play()')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [0]:
from zipfile import ZipFile 
file_name = "drive/My Drive/train.csv.zip"
  
# # opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zipF: 
    # printing all the contents of the zip file 
    zipF.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    data = zipF.extract('train.csv') 
    print('Done!')

file_name = "drive/My Drive/test.csv.zip"
  
# # opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zipF: 
    # printing all the contents of the zip file 
    zipF.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    data = zipF.extract('test.csv') 
    print('Done!')

File Name                                             Modified             Size
train.csv                                      2020-02-09 10:27:26     77009507
Extracting all the files now...
Done!
File Name                                             Modified             Size
test.csv                                       2020-02-09 10:27:26     24378147
Extracting all the files now...
Done!


In [0]:
import pandas as pd
train_df = pd.read_csv('train.csv', index_col=0)

from sklearn.model_selection import train_test_split
# Split test and train data
# df_train = train_df.fillna(train_df.median())


# df_train['opened_position_qty '].fillna((df_train['transacted_qty']), inplace=True)

# df_train['closed_position_qty'].fillna((0), inplace=True)
df_train = train_df.fillna(train_df.median())

# df_train['closed_position_qty'].fillna((df_train['transacted_qty'] / 2), inplace=True)


df_train['diff'] = (df_train['last_price'] - df_train['mid'])
df_train['avg_bid'] = (df_train['bid1'] * df_train['bid1vol'] + df_train['bid2'] * df_train['bid2vol'] + df_train['bid3'] * df_train['bid3vol'] + df_train['bid4'] * df_train['bid4vol'] + df_train['bid5'] * df_train['bid5vol']) / (df_train['bid1vol'] + df_train['bid2vol'] + df_train['bid3vol'] + df_train['bid4vol'] + df_train['bid5vol'])
df_train['avg_ask'] = (df_train['ask1'] * df_train['ask1vol'] + df_train['ask2'] * df_train['ask2vol'] + df_train['ask3'] * df_train['ask3vol'] + df_train['ask4'] * df_train['ask4vol'] + df_train['ask5'] * df_train['ask5vol']) / (df_train['ask1vol'] + df_train['ask2vol'] + df_train['ask3vol'] + df_train['ask4vol'] + df_train['ask5vol'])
df_train['diff_avg_bid'] = (df_train['mid'] - df_train['avg_bid'])
df_train['diff_avg_ask'] = (df_train['mid'] - df_train['avg_ask'])


# df_train['effective_spread'] = ((df_train['last_price'] - df_train['mid']) / df_train['mid'] )
# df_train['quoted_spread'] = (100 * (df_train['ask1'] - df_train['bid1']) / df_train['mid'] )
#df_train['diff_avg_bid_ask'] = (df_train['avg_ask'] - df_train['avg_bid'])
#X = df_train.drop('y', axis=1)


# X = df_train.drop(['id', 'y', 'bid1', 'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2','ask3','ask4','ask5'], axis=1)

y = df_train['y']


X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=0)



In [0]:
X_train.columns


Index(['last_price', 'mid', 'opened_position_qty ', 'closed_position_qty',
       'transacted_qty', 'd_open_interest', 'bid1vol', 'bid2vol', 'bid3vol',
       'bid4vol', 'bid5vol', 'ask1vol', 'ask2vol', 'ask3vol', 'ask4vol',
       'ask5vol', 'diff', 'avg_bid', 'avg_ask', 'diff_avg_bid',
       'diff_avg_ask'],
      dtype='object')

In [0]:

df = pd.DataFrame() 
df['ab'] = X_train['avg_bid']
df['aa'] = X_train['avg_ask']
df['dab'] = X_train['diff_avg_bid']
df['daa'] = X_train['diff_avg_ask']
df['diff'] = X_train['diff']



df['best_offer_vol_diff'] = X_train['bid1vol'] - X_train['ask1vol']
df['avg_best_offer'] = (X_train['bid1'] * X_train['bid1vol'] + X_train['ask1'] * X_train['ask1vol']) / (X_train['bid1vol'] + X_train['ask1vol'])
df['mid_avg_best_offer_diff'] = X_train['mid'] - df['avg_best_offer']
df['last_price_avg_best_offer_diff'] = X_train['last_price'] - df['avg_best_offer']




df_test = pd.DataFrame() 
df_test['ab'] = X_test['avg_bid']
df_test['aa'] = X_test['avg_ask']
df_test['dab'] = X_test['diff_avg_bid']
df_test['daa'] = X_test['diff_avg_ask']
df_test['diff'] = X_test['diff']


df_test['best_offer_vol_diff'] = X_test['bid1vol'] - X_test['ask1vol']
df_test['avg_best_offer'] = (X_test['bid1'] * X_test['bid1vol'] + X_test['ask1'] * X_test['ask1vol']) / (X_test['bid1vol'] + X_test['ask1vol'])
df_test['mid_avg_best_offer_diff'] = X_test['mid'] - df_test['avg_best_offer']
df_test['last_price_avg_best_offer_diff'] = X_test['last_price'] - df_test['avg_best_offer']





In [0]:
df = df.drop(['aa','ab','avg_best_offer'],axis=1)

In [0]:

df_test = df_test.drop(['aa','ab','avg_best_offer'],axis=1)


In [0]:
df

,dab,daa,diff,best_offer_vol_diff,mid_avg_best_offer_diff,last_price_avg_best_offer_diff
id,,,,,,
87249,0.588889,-0.564865,0.1,-7,-0.077778,0.022222
184693,0.598462,-1.007317,0.1,6,0.075000,0.175000
571316,0.393333,-0.530000,-0.3,1,0.033333,-0.266667
146937,1.085714,-0.485714,0.1,-3,-0.060000,0.040000
434363,0.918182,-0.440000,0.1,-3,-0.060000,0.040000
...,...,...,...,...,...,...
359783,0.314286,-0.525000,-0.1,5,0.055556,-0.044444
152315,0.640000,-0.627273,-0.2,-5,-0.142857,-0.342857
117952,0.600000,-0.588889,0.1,0,0.000000,0.100000


In [0]:
from sklearn.ensemble import GradientBoostingClassifier,VotingClassifier,RandomForestClassifier,AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [0]:
clf1 = GradientBoostingClassifier()
clf2 = RandomForestClassifier()
# clf3 = GaussianNB()
clf4 = AdaBoostClassifier()
clf5 = LGBMClassifier()
clf6 = XGBClassifier()
eclf1 = VotingClassifier(estimators=[('gb', clf1), ('rf', clf2), ('ada', clf4),('lgbm', clf5),('xgb', clf6)], voting='soft')
eclf1 = eclf1.fit(df, y_train)

In [0]:
eclf1.score(df_test,y_test)

0.6633326580910902

In [0]:
y_probs = eclf1.predict_proba(df_test)

In [0]:
y_probs = eclf1.predict_proba(df_test)
from sklearn.metrics import  roc_auc_score

roc_auc_score(y_test,y_probs[:,1])

0.656703223162583

In [0]:
len(y)

592380

# TEsting

In [0]:
import pandas as pd
test_df = pd.read_csv('test.csv')

# from sklearn.model_selection import test_test_split
df_test = test_df.fillna(test_df.median())
df_test['diff'] = (df_test['last_price'] - df_test['mid'])
df_test['avg_bid'] = (df_test['bid1'] * df_test['bid1vol'] + df_test['bid2'] * df_test['bid2vol'] + df_test['bid3'] * df_test['bid3vol'] + df_test['bid4'] * df_test['bid4vol'] + df_test['bid5'] * df_test['bid5vol']) / (df_test['bid1vol'] + df_test['bid2vol'] + df_test['bid3vol'] + df_test['bid4vol'] + df_test['bid5vol'])
df_test['avg_ask'] = (df_test['ask1'] * df_test['ask1vol'] + df_test['ask2'] * df_test['ask2vol'] + df_test['ask3'] * df_test['ask3vol'] + df_test['ask4'] * df_test['ask4vol'] + df_test['ask5'] * df_test['ask5vol']) / (df_test['ask1vol'] + df_test['ask2vol'] + df_test['ask3vol'] + df_test['ask4vol'] + df_test['ask5vol'])
df_test['diff_avg_bid'] = (df_test['mid'] - df_test['avg_bid'])
df_test['diff_avg_ask'] = (df_test['mid'] - df_test['avg_ask'])

df = pd.DataFrame() 
df['ab'] = df_test['avg_bid']
df['aa'] = df_test['avg_ask']
df['dab'] = df_test['diff_avg_bid']
df['daa'] = df_test['diff_avg_ask']
df['diff'] = df_test['diff']


df['best_offer_vol_diff'] = df_test['bid1vol'] - df_test['ask1vol']
df['avg_best_offer'] = (df_test['bid1'] * df_test['bid1vol'] + df_test['ask1'] * df_test['ask1vol']) / (df_test['bid1vol'] + df_test['ask1vol'])
df['mid_avg_best_offer_diff'] = df_test['mid'] - df['avg_best_offer']
df['last_price_avg_best_offer_diff'] = df_test['last_price'] - df['avg_best_offer']


df = df.drop(['aa','ab','avg_best_offer'],axis=1)

probbs = eclf1.predict_proba(df)
df_test['Predicted'] = probbs[:, 1]
header = ["id", "Predicted"]
df_test.to_csv('output.csv', columns = header,index=False)




In [0]:
df_test

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,bid4,bid5,ask1,ask2,ask3,ask4,ask5,bid1vol,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,diff,avg_bid,avg_ask,diff_avg_bid,diff_avg_ask,Predicted
0,592380,5178.4,5178.3,1.0,11.0,12.0,-10,5178.2,5178.0,5177.8,5177.6,5177.4,5178.4,5178.6,5178.8,5179.0,5179.8,3,16,3,1,1,1,4,1,5,2,1.000000e-01,5177.958333,5178.938462,0.341667,-0.638462,0.569495
1,592381,5133.0,5132.9,1.0,0.0,1.0,1,5132.8,5132.6,5132.4,5132.2,5132.0,5133.0,5133.2,5133.8,5134.0,5134.2,1,2,2,1,10,4,8,1,1,1,1.000000e-01,5132.187500,5133.306667,0.712500,-0.406667,0.248309
2,592382,5177.4,5178.0,2.0,0.0,2.0,2,5177.2,5176.8,5176.6,5176.4,5176.0,5178.8,5179.0,5179.2,5179.4,5179.6,8,18,1,1,6,7,4,1,2,5,-6.000000e-01,5176.735294,5179.136842,1.264706,-1.136842,0.424953
3,592383,5093.6,5093.9,0.0,2.0,2.0,-1,5093.2,5093.0,5092.8,5092.2,5092.0,5094.6,5094.8,5095.0,5095.2,5095.4,8,5,4,3,1,1,1,11,6,2,-3.000000e-01,5092.876190,5095.066667,1.023810,-1.166667,0.506372
4,592384,5189.2,5189.2,1.0,1.0,0.0,0,5188.8,5188.6,5188.4,5188.2,5188.0,5189.6,5190.6,5190.8,5191.0,5191.2,1,5,1,3,3,3,2,1,3,4,-1.818989e-12,5188.369231,5190.661538,0.830769,-1.461538,0.419954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191854,784234,5150.6,5150.8,1.0,1.0,0.0,0,5150.6,5150.4,5150.2,5150.0,5149.8,5151.0,5151.4,5151.6,5152.0,5152.4,1,5,3,4,2,1,3,1,3,2,-2.000000e-01,5150.186667,5151.760000,0.613333,-0.960000,0.474732
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,5121.8,5121.6,5122.8,5123.0,5123.2,5123.4,5123.6,3,1,5,1,20,16,17,4,1,16,-1.000000e-01,5121.833333,5123.140741,0.866667,-0.440741,0.224077
191856,784236,5192.8,5192.3,1.0,1.0,0.0,0,5192.0,5191.8,5191.2,5190.4,5190.2,5192.6,5192.8,5193.0,5193.2,5193.4,1,2,1,1,1,1,2,1,2,1,5.000000e-01,5191.233333,5193.000000,1.066667,-0.700000,0.470837
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,5151.2,5151.0,5152.0,5152.2,5152.4,5152.6,5152.8,4,5,6,8,37,7,1,2,1,1,1.000000e-01,5151.170000,5152.200000,0.730000,-0.300000,0.328646


In [0]:
check = pd.read_csv('output.csv')

In [0]:
check

,id,Predicted
0,592380,0.569495
1,592381,0.248309
2,592382,0.424953
3,592383,0.506372
4,592384,0.419954
...,...,...
191854,784234,0.474732
191855,784235,0.224077
191856,784236,0.470837
191857,784237,0.328646


In [0]:
c = pd.read_csv('output_firstensemble.csv')

In [0]:
import numpy as np
c['id'] = np.arange(592380,784239)

In [0]:
c

,id,Predicted
0,592380,0.528943
1,592381,0.383279
2,592382,0.406979
3,592383,0.415792
4,592384,0.376995
...,...,...
191854,784234,0.379256
191855,784235,0.209356
191856,784236,0.473233
191857,784237,0.391470


In [0]:
c.to_csv('output_first_with.csv')